In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip -q /content/drive/MyDrive/SB/Assignment1/train_data.zip -d /content/drive/MyDrive/SB/Assignment1/
# !unzip -q /content/drive/MyDrive/SB/Assignment1/true_test.zip -d /content/drive/MyDrive/SB/Assignment1/
# !unzip -q /content/drive/MyDrive/SB/Assignment1/test_YOLO_format.zip -d /content/drive/MyDrive/SB/Assignment1/
# !unzip -q /content/drive/MyDrive/SB/Assignment1/sharpened.zip -d /content/drive/MyDrive/SB/Assignment1/

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-141-ge8ef8fb torch 1.10.0+cu111 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.7/107.7 GB disk)


In [ ]:
# train model yolov5s on XXX epochs
!python train.py --img 480 --batch 16 --epochs 30 --data /content/drive/MyDrive/SB/Assignment1/custom_data.yaml --weights yolov5s.pt --cache

train: weights=yolov5m.pt, cfg=, data=/content/drive/MyDrive/SB/Assignment1/custom_data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=16, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-140-g19c56e6 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anc

In [ ]:
%cp -av "/content/yolov5/runs/train/exp4" "/content/drive/MyDrive/SB/Assignment1/results/results30"

'/content/yolov5/runs/train/exp3' -> '/content/drive/MyDrive/SB/Assignment1/results/results30'
'/content/yolov5/runs/train/exp3/hyp.yaml' -> '/content/drive/MyDrive/SB/Assignment1/results/results30/hyp.yaml'
'/content/yolov5/runs/train/exp3/opt.yaml' -> '/content/drive/MyDrive/SB/Assignment1/results/results30/opt.yaml'
'/content/yolov5/runs/train/exp3/events.out.tfevents.1639407000.9ffbdca5889e.4330.0' -> '/content/drive/MyDrive/SB/Assignment1/results/results30/events.out.tfevents.1639407000.9ffbdca5889e.4330.0'
'/content/yolov5/runs/train/exp3/labels_correlogram.jpg' -> '/content/drive/MyDrive/SB/Assignment1/results/results30/labels_correlogram.jpg'
'/content/yolov5/runs/train/exp3/labels.jpg' -> '/content/drive/MyDrive/SB/Assignment1/results/results30/labels.jpg'
'/content/yolov5/runs/train/exp3/train_batch0.jpg' -> '/content/drive/MyDrive/SB/Assignment1/results/results30/train_batch0.jpg'
'/content/yolov5/runs/train/exp3/train_batch1.jpg' -> '/content/drive/MyDrive/SB/Assignment1/re

In [ ]:
!python detect.py --save-txt --weights /content/drive/MyDrive/SB/Assignment1/results/resultsSharp50/weights/last.pt --img 480 --conf 0.25 --source /content/drive/MyDrive/SB/Assignment1/sharpened/images/test

detect: weights=['/content/drive/MyDrive/SB/Assignment1/results/resultsSharp50/weights/last.pt'], source=/content/drive/MyDrive/SB/Assignment1/sharpened/images/test, imgsz=[480, 480], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-141-ge8ef8fb torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/250 /content/drive/MyDrive/SB/Assignment1/sharpened/images/test/0001.png: 384x480 1 ear, Done. (0.191s)
image 2/250 /content/drive/MyDrive/SB/Assignment1/sharpened/images/test/0002.png: 384x480 1 ear, Done. (0.174s)
image 3/250 /content/drive/MyDrive/SB/Assignment1/sharpened/images/test/0003.png: 384x480 1 ear, Done. (0.165s)
i

In [ ]:
import cv2
import numpy as np
from sklearn.metrics import f1_score


class Evaluation:

    def convert2mask(self, mt, shape):
        # Converts coordinates of bounding-boxes into blank matrix with values set where bounding-boxes are.

        t = np.zeros([shape, shape])
        for m in mt:
            c, x, y, w, h = m
            cv2.rectangle(t, (x,y), (x+w, y+h), 1, -1)
        return t

    def prepare_for_detection(self, prediction, ground_truth):
            # For the detection task, convert Bounding-boxes to masked matrices (0 for background, 1 for the target). If you run segmentation, do not run this function

            if len(prediction) == 0:
                return [], []

            # Large enough size for base mask matrices:
            shape = 2*max(np.max(prediction), np.max(ground_truth)) 
            
            p = self.convert2mask(prediction, shape)
            gt = self.convert2mask(ground_truth, shape)

            return p, gt

    def iou_compute(self, p, gt):
            # Computes Intersection Over Union (IOU)
            if len(p) == 0:
                return 0

            intersection = np.logical_and(p, gt)
            union = np.logical_or(p, gt)

            iou = np.sum(intersection) / np.sum(union)

            return iou

    def f1_score(self, p, gt):
        if len(p) == 0:
            return 0

        f1 = f1_score(gt.reshape((-1,)), p.reshape((-1,)), average='binary')

        return f1



In [ ]:
def convert_data(a):
  image_width = 480
  image_height = 360
  w = a[3] * image_width
  h = a[4] * image_height
  return [int(a[0]), int(round(a[1]*image_width - w/2)), int(round(a[2]*image_height - h/2)), int(round(w)), int(round(h))]

def make_list(lines, convert):
  converted_list = []
  for line in lines:
    if convert:
      converted_list.append(convert_data(list(map(float, line.split()))))
    else:
      converted_list.append(list(map(int, line.split())))
  
  return converted_list

def calculate_combined_iou(from_index, to_index):
  eval = Evaluation()
  iou_list = []
  f1_list = []
  for name in range(from_index, to_index+1):
    name_string = "000" + str(name)
    name_string = name_string[-4:]

    # if name_string in ['0109', '0118', '0119']:
    #   continue

    # if name_string in ['0015', '0118', '0119', '0217']:
    #   continue

    # if name_string in ['0118', '0119', '0109']:
    #    continue

    with open("/content/yolov5/runs/detect/exp5/labels/"+name_string+".txt", "r") as pred_file, open("/content/drive/MyDrive/SB/Assignment1/test_YOLO_format/"+name_string+".txt", "r") as annot_file:
      prediction_list = make_list(pred_file.readlines(), convert = True)
      annotation_list = make_list(annot_file.readlines(), convert = False)

      p, gt = eval.prepare_for_detection(prediction_list, annotation_list)
      iou = eval.iou_compute(p, gt)
      iou_list.append(iou)
      f1 = eval.f1_score(p, gt)
      f1_list.append(f1)
  
  combined_iou = np.average(iou_list)
  print("Average IOU:", f"{combined_iou:.2%}")

  combined_f1 = np.average(f1_list)
  print("Average F1:", f"{combined_f1:.2}")


In [ ]:
calculate_combined_iou(1, 250)

Average IOU: 89.66%
Average F1: 0.94
